<div align="center">

# <span style="color: #3498db;">CA2 - Genetic Algorithm</span>

**<span style="color:rgb(247, 169, 0);">Parsa Saeednia</span> - <span style="color:rgb(143, 95, 195);">810102460</span>**

</div>


<div style="font-family: Arial, sans-serif; line-height: 1.6;">

### 📊 Matplotlib – Data Visualization in Python  

matplotlib is a python library that is mainly used for data visualization. This library allows you to plot different type of figures including scatters and histograms. In the first part of this project you are supposed to implement a genetic algorithm. To visualize plots that are required in the project description use plotting as much as you can because it gives a great insight on what is happening during each run. It also helps you to compare your results whenevever you want to understand effect of different parameters during different runs.
For more information, check [this notebook](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/04.00-Introduction-To-Matplotlib.ipynb) and visit [the website](https://matplotlib.org/stable/tutorials/pyplot.html#sphx-glr-tutorials-pyplot-py).

In [374]:
import matplotlib.pyplot as plt

# <span style="color: #3498db;">Genetic Algorithm</span>

In [375]:
import random
import numpy as np

In [376]:
def fourierCalculate(constantsMap: dict, x):
    result = constantsMap["a0"] / 2
    
    for i in range(1, len(constantsMap) // 2 + 1):
        result += constantsMap[f"a{i}"] * np.cos(i * x)
        result += constantsMap[f"b{i}"] * np.sin(i * x)
    
    return result

In [377]:
class Database:
    def __init__(self, populationSize_, generationsCount_, coeffsNum_, mutationRate_, coeffsRange_, mutationRange_):
        self.populationSize = populationSize_
        self.generationsCount = generationsCount_
        self.coeffsNum = coeffsNum_
        self.mutationRate = mutationRate_
        self.coeffsRange = coeffsRange_
        self.mutationRange = mutationRange_
        
    
    def getPopulationSize(self):
        return self.populationSize
        
        
    def getGenerationCount(self):
        return self.generationsCount
    
    
    def getCoeffsNum(self):
        return self.coeffsNum
        
        
    def getMutationRate(self):
        return self.mutationRate
    
    
    def getCoeffsRange(self):
        return self.coeffsRange
    
    
    def getTSamples(self):
        return self.tSamples
    
    
    def getFSamples(self):
        return self.fSamples
    
    
    def getPopulation(self):
        return self.population
    
    
    def getMutationRange(self):
        return self.mutationRange
    
    
    def setMutationRange(self, mutationRange_):
        self.mutationRange = mutationRange_
        
        
    def setFSamples(self, fSamples_):
        self.fSamples = fSamples_
        
    
    def setTSamples(self, tSamples_):
        self.tSamples = tSamples_
        
    
    def setCoeffsRange(self, coeffsRange_):
        self.coeffsRange = coeffsRange_
        
        
    def setPopulationSize(self, populationSize_):
        self.populationSize = populationSize_
        
        
    def setGenerationCount(self, generationsCount_):
        self.generationsCount = generationsCount_
        
        
    def setCoeffsNum(self, coeffsNum_):
        self.coeffsNum = coeffsNum_
        
        
    def setMutationRate(self, mutationRate_):
        self.mutationRate = mutationRate_
        
        
    def setPopulation(self, population_):
        self.population = population_
    

In [378]:
class Species:
    def __init__(self, database: Database, genome_):
        if genome_ == None:
            self.genome = self.generateGenome(database)
        else:
            self.genome = genome_
            
        self.fourierEstimations = np.array([fourierCalculate(self.genome, val) for val in database.getTSamples()])

    
    def generateGenome(self, database: Database):
        genome = dict()
        coeffsRange = database.getCoeffsRange()
        genome["a0"] = random.uniform(coeffsRange[0], coeffsRange[1])
        for i in range(1, database.getCoeffsNum() // 2 + 1):
            genome[f"a{i}"] = random.uniform(coeffsRange[0], coeffsRange[1])
            genome[f"b{i}"] = random.uniform(coeffsRange[0], coeffsRange[1])
            
        return genome
            
            
    def rmseFitness(self, database: Database):
        rmse = np.sqrt(np.mean((self.fourierEstimations - database.getFSamples()) ** 2))
        return rmse
    
    
    def maeFitness(self, database: Database):
        true_values = database.getFSamples()
        mae = np.sum(np.abs(self.fourierEstimations - true_values))
        mae /= len(true_values)
        return mae
    
    
    def rtwoFitness(self, database: Database):
        true_values = database.getFSamples()
        rtwo = np.sum((true_values - self.fourierEstimations) ** 2) / np.sum((true_values - np.mean(true_values)) ** 2)
        rtwo = 1 - rtwo
        return rtwo
    
    
    def mergeSinglePoint(self, other: "Species"):
        crossoverPoint = random.randint(1, len(self.genome) - 1)
        keys = list(self.genome.keys())

        firstOffspring = {key: self.genome[key] if i < crossoverPoint else other.genome[key] for i, key in enumerate(keys)}
        secondOffspring = {key: other.genome[key] if i < crossoverPoint else self.genome[key] for i, key in enumerate(keys)}

        return firstOffspring, secondOffspring

    
    
    def mergeMultiPoint(self, other: "Species", pointsCount):
        crossover_points = sorted(random.sample(range(1, len(self.genome)), pointsCount))
        firstOffspring, secondOffspring = {}, {}

        keys = list(self.genome.keys())
        
        toggle = True
        for i, key in enumerate(keys):
            if i in crossover_points:
                toggle = not toggle  # Switch parent at crossover points
            firstOffspring[key] = self.genome[key] if toggle else other.genome[key]
            secondOffspring[key] = other.genome[key] if toggle else self.genome[key]

        return firstOffspring, secondOffspring
        
        
    def mergeUniform(self, other: "Species"):
        firstOffspring, secondOffspring = {}, {}

        for key in self.genome.keys():
            if random.random() < 0.5:
                firstOffspring[key] = self.genome[key]
                secondOffspring[key] = other.genome[key]
            else:
                firstOffspring[key] = other.genome[key]
                secondOffspring[key] = self.genome[key]

        return firstOffspring, secondOffspring
    
    def mutate(self, database: Database):
        mutationRange = database.getMutationRange()
        coeffsRange = database.getCoeffsRange()
        
        for key in self.genome.keys():
            if random.random() < database.getMutationRate():
                self.genome[key] += random.uniform(mutationRange[0], mutationRange[1])
                self.genome[key] = np.clip(self.genome[key], coeffsRange[0], coeffsRange[1])
        

In [379]:
class NaturalSelection:
    def __init__(self, database_: Database, fitnessFunction_, selectionType_, mergeType_):
        self.database = database_
        self.generatePopulation()
        
        self.fitnessFunction = fitnessFunction_
        self.selectionType = selectionType_
        self.mergeType = mergeType_
        
        self.evolve()
            
            
    def generatePopulation(self):
        species = []
        for i in range(self.database.getPopulationSize()):
            species.append(Species(self.database, None))
            
        self.database.setPopulation(species)
        
        
    def evolve(self):
        self.generationFitness = dict()
        
        for i in range(self.database.getGenerationCount()):
            population = self.database.getPopulation()
            # population = sorted(population, key=lambda s: self.fitnessFunction(s, self.database))
            bestSpecies = min(population, key=lambda s: self.fitnessFunction(s, self.database))
            self.generationFitness[i] = self.fitnessFunction(bestSpecies, self.database)
            print(f"{i} :", self.generationFitness[i])
            newGeneration = self.crossover()
            newGeneration = self.mutation(newGeneration)
            self.database.setPopulation(newGeneration)
            
            
    def sampleRandom(self):
        sample = random.sample(self.database.getPopulation(), self.selectionType["size"])
        return sample
        
        
    def tournamentSelect(self):
        sample = self.sampleRandom()
        sample = sorted(sample, key=lambda s: self.fitnessFunction(s, self.database))
        return (sample[0], sample[1])
    
    
    def rankSelect(self):
        population = self.database.getPopulation()
        population = sorted(population, key=lambda s: self.fitnessFunction(s, self.database), reverse=True)
        populationSize = self.database.getPopulationSize()
        
        ranks = np.arange(1, populationSize + 1)
        probabilities = ((2 - self.selectionType["selectionPressure"]) / populationSize + 
                         (2 * ranks * (self.selectionType["selectionPressure"] - 1)) / (populationSize * (populationSize - 1)))
        probabilities /= sum(probabilities)
        
        sample = np.random.choice(population, size=2, replace=False, p=probabilities)
        return (sample[0], sample[1])
    
    
    def rouletteWheelSelect(self):
        population = self.database.getPopulation()
        fitnessValues = np.array([self.fitnessFunction(species, self.database) for species in population])

        probabilities = fitnessValues / np.sum(fitnessValues)
        cumulative_probs = np.cumsum(probabilities)

        sample = []
        for _ in range(2):
            random_value = np.random.rand()
            selected_index = np.where(cumulative_probs >= random_value)[0][0]
            sample.append(population[selected_index])
        
        return tuple(sample)
            
        
    def crossover(self):
        newGeneration = []
        for _ in range(self.database.getPopulationSize() // 2):
            parents = self.selectionType["function"](self)

            if "pointsCount" in self.mergeType:
                offspringsData = self.mergeType["function"](parents[0], parents[1], self.mergeType["pointsCount"])
            else:
                offspringsData = self.mergeType["function"](parents[0], parents[1])
            
            offsprings = [Species(self.database, offspringData) for offspringData in offspringsData]
                
            newGeneration.extend(offsprings)
            
        return newGeneration

    
    def mutation(self, newGeneration : list[Species]):
        for species in newGeneration:
            species.mutate(self.database)
        
        return newGeneration
        

In [380]:
# These functions are given as samples to use in the algorithm
def getTargetFunction(functionName="sin_cos"):
    def sinCosFunction(t):
        """Target function: sin(2πt) + 0.5*cos(4πt)."""
        return np.sin(2 * np.pi * t) + 0.5 * np.cos(4 * np.pi * t)

    def linearFunction(t):
        """Simple linear function: y = 2t + 1."""
        return 2 * t + 1

    def quadraticFunction(t):
        """Quadratic function: y = 4t^2 - 4t + 2."""
        return 4 * (t**2) - 4 * t + 2

    def cubicFunction(t):
        """Cubic function: y = 8t^3 - 12t^2 + 6t."""
        return 8 * (t**3) - 12 * (t**2) + 6 * t

    def gaussianFunction(t):
        """Gaussian function centered at t=0.5."""
        mu = 0.5
        sigma = 0.1  # Adjust sigma to control the width of the peak
        return np.exp(-((t - mu) ** 2) / (2 * sigma**2))

    def squareWaveFunction(t):
        """Approximation of a square wave. Smoothed for better Fourier approximation."""
        return 0.5 * (np.sign(np.sin(2 * np.pi * t)) + 1)

    def sawtoothFunction(t):
        """Sawtooth wave, normalized to [0, 1]."""
        return (t * 5) % 1

    def complexFourierFunction(t):
        return (
            np.sin(2 * np.pi * t)
            + 0.3 * np.cos(4 * np.pi * t)
            + 0.2 * np.sin(6 * np.pi * t)
            + 0.1 * np.cos(8 * np.pi * t)
        )

    def polynomialFunction(t):
        return 10 * (t**5) - 20 * (t**4) + 15 * (t**3) - 4 * (t**2) + t + 0.5

    functionOptions = {
        "sin_cos": sinCosFunction,
        "linear": linearFunction,
        "quadratic": quadraticFunction,
        "cubic": cubicFunction,
        "gaussian": gaussianFunction,
        "square_wave": squareWaveFunction,
        "sawtooth": sawtoothFunction,
        "complex_fourier": complexFourierFunction,
        "polynomial": polynomialFunction,
    }

    selectedFunction = functionOptions.get(functionName.lower())
    if selectedFunction:
        return selectedFunction

In [381]:
# algorithm parameters
numCoeffs = 41
populationSize = 100
generations = 200
mutationRate = 0.2
mutationRange = (-0.25, 0.25)
functionRange = (-np.pi, np.pi)
sampleCount = 100
coeffsRange = (-10, 10)

In [382]:
fitnessMap = {"RootedMeanSquaredError" : Species.rmseFitness, 
              "MeanAbsoluteError" : Species.maeFitness, 
              "R2CoefficientOfDetermination" : Species.rtwoFitness}

selectionMap = {"TournamentSelection" : {"function" : NaturalSelection.tournamentSelect, "size" : 7}, 
                "RankingSelection" : {"function" : NaturalSelection.rankSelect, "selectionPressure" : 1.25}, 
                "rouletteWheelSelect" : {"function" : NaturalSelection.rouletteWheelSelect}}

crossoverMap = {"SinglePoint" : {"function" : Species.mergeSinglePoint}, 
                "MultiPoint" : {"function" : Species.mergeMultiPoint, "pointsCount" : 8}, 
                "Uniform" : {"function" : Species.mergeUniform}}

In [383]:
# generate samples
tSamples = np.linspace(functionRange[0], functionRange[1], sampleCount)
fSamples = getTargetFunction("sin_cos")(tSamples)
database = Database(populationSize, generations, numCoeffs, mutationRate, coeffsRange, mutationRange)
database.setTSamples(tSamples)
database.setFSamples(np.array(fSamples))
naturalSelection = NaturalSelection(database, 
                                    fitnessMap["RootedMeanSquaredError"], 
                                    selectionMap["TournamentSelection"], 
                                    crossoverMap["Uniform"])


0 : 20.52871697099331
1 : 19.287416130973565
2 : 18.234358530845967
3 : 16.963650868391586
4 : 15.51605750635036
5 : 13.238401627150406
6 : 13.316722073765627
7 : 12.406730982495066
8 : 10.236855233364444
9 : 9.668607664367125
10 : 9.327737710293041
11 : 8.820919138884427
12 : 8.316840924717955
13 : 7.4899167005943
14 : 7.222832112628176
15 : 6.692273839920998
16 : 6.165043811253211
17 : 6.179475239437374
18 : 5.735879671411115
19 : 5.361794843013634
20 : 5.320662244161701
21 : 5.057701723643451
22 : 4.990171765335191
23 : 4.931944574612602
24 : 4.81855511747612
25 : 4.605910237507889
26 : 4.529817332337599
27 : 4.489898469734003
28 : 4.296784645523988
29 : 4.256090246978786
30 : 4.21733664310392
31 : 4.109241075068704
32 : 3.9544194357751006
33 : 3.832247414152518
34 : 3.707797745040409
35 : 3.6267903649564
36 : 3.582865227101635
37 : 3.4642577301626702
38 : 3.2522353548797205
39 : 3.2434360074720536
40 : 3.112246418897507
41 : 3.024108420341236
42 : 2.956359436497166
43 : 2.878805397